In [124]:
from pyspark.sql import SparkSession

In [125]:
from pyspark.sql.functions import *

In [126]:
from pyspark.sql.types import *

In [127]:
spark=SparkSession.builder.master("local").appName("mock").getOrCreate()

In [128]:
network=spark.read.csv("/home/labuser/Downloads/network.csv",header=True)

In [129]:
network.show()

+---------+-----------+------------+
|series_id|series_info|     network|
+---------+-----------+------------+
|      101|       null|     netflix|
|      102|       null|amazon_prime|
|      103|       null|     appletv|
|      104|       null|     netflix|
|      105|       null|         HBO|
|      106|       null|         HBO|
|      107|       null|amazon_prime|
|      108|       null|     netflix|
|      109|       null|         ted|
|      110|       null|         sky|
+---------+-----------+------------+


In [130]:
series=spark.read.csv("/home/labuser/Downloads/series.csv",header=True)

In [131]:
series.show()

+---------+-------------+---------------+------+
|series_id|  series_name|          genre|rating|
+---------+-------------+---------------+------+
|      101| rickandmorty|          scifi|   9.1|
|      102|   invincible|       animated|   8.7|
|      103|forallmankind|historicalscifi|   7.9|
|      104|  blackmirror|  thrillerscifi|   8.0|
|      105|    westworld|          scifi|   8.1|
+---------+-------------+---------------+------+


In [132]:
con=series.withColumn("combine",concat_ws(" ",series.series_name,series.genre))

In [133]:
con.show()

+---------+-------------+---------------+------+--------------------+
|series_id|  series_name|          genre|rating|             combine|
+---------+-------------+---------------+------+--------------------+
|      101| rickandmorty|          scifi|   9.1|  rickandmorty scifi|
|      102|   invincible|       animated|   8.7| invincible animated|
|      103|forallmankind|historicalscifi|   7.9|forallmankind his...|
|      104|  blackmirror|  thrillerscifi|   8.0|blackmirror thril...|
|      105|    westworld|          scifi|   8.1|     westworld scifi|
+---------+-------------+---------------+------+--------------------+


In [134]:
ser1=con.select("series_id",'combine','rating')

In [135]:
ser1.show()

+---------+--------------------+------+
|series_id|             combine|rating|
+---------+--------------------+------+
|      101|  rickandmorty scifi|   9.1|
|      102| invincible animated|   8.7|
|      103|forallmankind his...|   7.9|
|      104|blackmirror thril...|   8.0|
|      105|     westworld scifi|   8.1|
+---------+--------------------+------+


In [136]:
fil=network.join(ser1,on='series_id',how='inner')

In [137]:
fil.show()

+---------+-----------+------------+--------------------+------+
|series_id|series_info|     network|             combine|rating|
+---------+-----------+------------+--------------------+------+
|      101|       null|     netflix|  rickandmorty scifi|   9.1|
|      102|       null|amazon_prime| invincible animated|   8.7|
|      103|       null|     appletv|forallmankind his...|   7.9|
|      104|       null|     netflix|blackmirror thril...|   8.0|
|      105|       null|         HBO|     westworld scifi|   8.1|
+---------+-----------+------------+--------------------+------+


In [138]:
new=fil.withColumn('series_info',coalesce(fil.series_info,fil.combine))

In [139]:
new.show()

+---------+--------------------+------------+--------------------+------+
|series_id|         series_info|     network|             combine|rating|
+---------+--------------------+------------+--------------------+------+
|      101|  rickandmorty scifi|     netflix|  rickandmorty scifi|   9.1|
|      102| invincible animated|amazon_prime| invincible animated|   8.7|
|      103|forallmankind his...|     appletv|forallmankind his...|   7.9|
|      104|blackmirror thril...|     netflix|blackmirror thril...|   8.0|
|      105|     westworld scifi|         HBO|     westworld scifi|   8.1|
+---------+--------------------+------------+--------------------+------+


In [140]:
new1=new.select("series_id","series_info","network","rating")

In [141]:
new1.show()

+---------+--------------------+------------+------+
|series_id|         series_info|     network|rating|
+---------+--------------------+------------+------+
|      101|  rickandmorty scifi|     netflix|   9.1|
|      102| invincible animated|amazon_prime|   8.7|
|      103|forallmankind his...|     appletv|   7.9|
|      104|blackmirror thril...|     netflix|   8.0|
|      105|     westworld scifi|         HBO|   8.1|
+---------+--------------------+------------+------+


In [142]:
high=new1.orderBy("rating",ascending=False)

In [143]:
high.show()

+---------+--------------------+------------+------+
|series_id|         series_info|     network|rating|
+---------+--------------------+------------+------+
|      101|  rickandmorty scifi|     netflix|   9.1|
|      102| invincible animated|amazon_prime|   8.7|
|      105|     westworld scifi|         HBO|   8.1|
|      104|blackmirror thril...|     netflix|   8.0|
|      103|forallmankind his...|     appletv|   7.9|
+---------+--------------------+------------+------+


In [144]:
high.select("series_id").first()[0]

'101'

In [145]:
s=series.select(sum("rating")).collect()[0][0]

In [146]:
s1=int(s)

In [147]:
s1

41

In [148]:
s1=float(s)

In [149]:
s1

41.8